In [93]:
import zipfile
import os
from os import walk
from os.path import join
import pandas as pd

In [94]:
from google.colab import files
uploaded = files.upload()


Saving SpamData.zip to SpamData (1).zip


In [95]:
zip_file = 'SpamData.zip'
unzip_dir = '/content/DataExtracted'

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)


In [96]:
import os
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: /content


**Constants**

In [71]:
EXAMPLE_FILE = 'practice_email.txt'

SPAM_1_PATH = 'DataExtracted/SpamData/01_Processing/spam_assassin_corpus/spam_1'
SPAM_2_PATH = 'DataExtracted/SpamData/01_Processing/spam_assassin_corpus/spam_2'
EASY_NONSPAM_1_PATH = 'DataExtracted/SpamData/01_Processing/spam_assassin_corpus/easy_ham_1'
EASY_NONSPAM_2_PATH = 'DataExtracted/SpamData/01_Processing/spam_assassin_corpus/easy_ham_2'

SPAM_CAT = 1
HAM_CAT = 0

In [72]:
stream = open(EXAMPLE_FILE, encoding='utf-8')
message = stream.read()
stream.close()

print(message)

From exmh-workers-admin@redhat.com  Thu Aug 22 12:36:23 2002
Return-Path: <exmh-workers-admin@spamassassin.taint.org>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36
	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:36:16 +0100 (IST)
Received: from listman.spamassassin.taint.org (listman.spamassassin.taint.org [66.187.233.211]) by
    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MBYrZ04811 for
    <zzzz-exmh@spamassassin.taint.org>; Thu, 22 Aug 2002 12:34:53 +0100
Received: from listman.spamassassin.taint.org (localhost.localdomain [127.0.0.1]) by
    listman.redhat.com (Postfix) with ESMTP id 8386540858; Thu, 22 Aug 2002
    07:35:02 -0400 (EDT)
Delivered-To: exmh-workers@listman.spamassassin.taint.org
Received: from int-mx1.corp

In [73]:
stream = open(EXAMPLE_FILE, encoding='utf-8')
is_body = False
lines = []

for line in stream:
  if is_body:
    lines.append(line)
  elif line == '\n':
    is_body = True

stream.close()
email_body = '\n'.join(lines)

print(email_body)




Dear Mr Still



Good tidings to you and all your staff for the festive season ahead (Christmas).

Now to the crux of the matter-in-hand: I am a fully qualified Santa Claus and am wondering whether you might consider me to run my own "Santa's Grotto" in your store.

But WAIT! You're probably thinking: "What makes him so special?"

Well, first of all, I have made several changes to the characterisation of Father Christmas. Rather than greeting the children with shouts of "Ho, ho, ho!" I prefer to whisper the phrase "Dependence is not unfathomable in this cruel world we live in". In addition, my gifts are ALL hand-made, ranging from felt hoops to vanilla-pod holders.

You will note also, from the enclosed sketch, that I have radically redesigned Santa's outfit and have renamed my character "Lord Buckles". Would you be interested in employing me? I promise NEVER to let you down.

I look forward to hearing from you.



Best wishes

Robin Cooper

[Excerpt from the book: The Timewaster Let

**Generator Functions**

In [74]:
def generate_squares(N):
  for my_number in range(N):
    yield my_number ** 2

In [75]:
for i in generate_squares(5):
  print(i, end=' ->')

0 ->1 ->4 ->9 ->16 ->

**Email body extraction**

In [76]:
def email_body_generator(path):

  for root, dirnames, filenames in walk(path):
    for file_name in filenames:

      filepath = join(root, file_name)
      stream = open(filepath, encoding='latin-1')
      is_body = False
      lines = []

      for line in stream:
        if is_body:
          lines.append(line)
        elif line == '\n':
          is_body = True

      stream.close()
      email_body = '\n'.join(lines)

      yield file_name, email_body

In [97]:
def df_from_directory(path,classification):
  rows = []
  row_names = []

  for file_name, email_body in email_body_generator(path):
    rows.append({'MESSAGE': email_body, 'CATEGORY': classification})
    row_names.append(file_name)


  return pd.DataFrame(rows, index=row_names)

In [98]:
spam_emails = df_from_directory(SPAM_1_PATH, 1)
spam_emails = pd.concat([spam_emails, df_from_directory(SPAM_2_PATH, 1)])
spam_emails.head()

,MESSAGE,CATEGORY
00333.4bb36a535cb3d738f30f985f1e10a786,------000000000000000000000\n\nContent-Type: t...,1
00160.cec5f611ae665ff0add6c4928d47f2be,FREE 30 day supply of HGH 1000:\n\nLook Younge...,1
00240.2ff7f745285653a238214d975859406b,\n\nDear Sir or Madam\n\n\n\nIn the past you h...,1
00438.41295e1df4b651b7611316331b8468e4,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",1
00425.1434e0ab4e5235b64825b4c2a0999d76,<HTML><HEAD><TITLE>Hi i'm Rita !!!</TITLE>\n\n...,1


In [99]:
spam_emails.shape

(1898, 2)

In [100]:
ham_emails = df_from_directory(EASY_NONSPAM_1_PATH, 0)
ham_emails = pd.concat([ham_emails, df_from_directory(EASY_NONSPAM_2_PATH, 0)])
ham_emails.shape

(3901, 2)

In [101]:
data = pd.concat([spam_emails, ham_emails])
print('Shape of entire DataFrame is:', data.shape)
data.head()

Shape of entire DataFrame is: (5799, 2)


,MESSAGE,CATEGORY
00333.4bb36a535cb3d738f30f985f1e10a786,------000000000000000000000\n\nContent-Type: t...,1
00160.cec5f611ae665ff0add6c4928d47f2be,FREE 30 day supply of HGH 1000:\n\nLook Younge...,1
00240.2ff7f745285653a238214d975859406b,\n\nDear Sir or Madam\n\n\n\nIn the past you h...,1
00438.41295e1df4b651b7611316331b8468e4,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",1
00425.1434e0ab4e5235b64825b4c2a0999d76,<HTML><HEAD><TITLE>Hi i'm Rita !!!</TITLE>\n\n...,1


In [102]:
data.tail()

,MESSAGE,CATEGORY
00719.d3f6422685d691cd7d0aea8851b831f9,--==_Exmh_1737280444P\n\nContent-Type: text/pl...,0
00566.dbc5fe0b052e8d60b7a4e1d40f0d9652,Bob Proulx <bob@proulx.com> wrote:\n\nTo: <raz...,0
01060.95d3e0a8c47b33d1533f18ac2c60c81a,NRI Number One\n\n\n\nExpats help Hindi films ...,0
00789.f5ae0a050a9e179558e9d351d9cfb7d2,--==_Exmh_1217613150P\n\nContent-Type: text/pl...,0
00555.219cc078f109a6eea18c579f143a8cef,\n\n I hate to ask...I've heard it mentione...,0


**Data Cleaning: Checking for missing values**

In [103]:
#Check if any message bodies are null
#data.MESSAGE
#second option
data['MESSAGE'].isnull().values.any()


False

In [104]:
#check if there are empty emails(string length zero)
(data.MESSAGE.str.len() == 0).any()

True

In [85]:
(data.MESSAGE.str.len() == 0).sum()

3

In [86]:
data.MESSAGE.isnull().sum()

0

**Locate empty emails**

In [105]:
type(data.MESSAGE.str.len() == 0)

pandas.core.series.Series

In [106]:
data[data.MESSAGE.str.len() == 0].index

Index(['cmds', 'cmds', 'cmds'], dtype='object')

In [107]:
data.index.get_loc('cmds')

array([False, False, False, ..., False, False, False])

**Remove System File Entries from Dataframe**

In [92]:
type(data)


NoneType

**Add Document Ids to track emails in dataset**